In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
import re
import os

In [3]:
def blast_and_filter(gene_names, e_value_threshold=0.005, percent_identity_threshold=90, coverage_threshold=70):
    """
    Performs BLAST search for given genes and filters results based on thresholds.

    Inputs:
        gene_names (list of str): List of gene names to process.
        e_value_threshold (float): Maximum E-value to consider an alignment (default: 0.01).
        percent_identity_threshold (float): Minimum percentage identity for inclusion (default: 30).
        coverage_threshold (float): Minimum query coverage percentage for inclusion (default: 30).

    Outputs:
        - Filtered BLAST results in FASTA format (saved in blast_results/{gene_name}_blast.fasta)
        - Summary of alignments printed to console
    """
    if not os.path.exists("blast_results"):
        os.makedirs("blast_results")
        
    for name_gene in gene_names:
        # Read sequence and run BLAST
        try:
            if not os.path.exists("genes"):
                os.makedirs("genes")
        
            query_seq = SeqIO.read(f"genes/{name_gene}.fasta", "fasta")
        except FileNotFoundError:
            print(f"File not found for {name_gene}. Skipping...")
            continue

        print(f"Starting BLAST for {name_gene}...")
        result_handle = NCBIWWW.qblast("blastp", "nr", query_seq.seq)
        print(f"BLAST completed for {name_gene}.")

        # Parse and filter results
        blast_records = NCBIXML.parse(result_handle)
        output_path = f"blast_results/{name_gene}_blast.fasta"
        
        with open(output_path, "w") as output_handle:
            total_alignments = 0
            saved_alignments = 0
            
            for blast_record in blast_records:
                total_alignments = len(blast_record.alignments)
                print(f"Number of alignments found for {name_gene}: {total_alignments}")
                
                for i, alignment in enumerate(blast_record.alignments, start=1):
                    for hsp in alignment.hsps:
                        query_cover = (hsp.align_length / blast_record.query_letters) * 100
                        percent_identity = (hsp.identities / hsp.align_length) * 100
                        
                        # Displays the alignment in the terminal
                        species_match = re.search(r"\[(.*?)\]", alignment.title)
                        species = species_match.group(1) if species_match else "Unknown species"
                        
                        # Applies the filters
                        if (hsp.expect <= e_value_threshold and
                            percent_identity >= percent_identity_threshold and
                            query_cover >= coverage_threshold):
                            
                            # Writes to the file if the alignment is filtered
                            SeqIO.write(
                                SeqIO.SeqRecord(
                                    seq=Seq(hsp.sbjct),  # Converta hsp.sbjct para um objeto Seq
                                    id=f"{species} | {alignment.accession}",  # Coloca a espécie primeiro
                                    description=f"E-value: {hsp.expect:.2e}, Identities: {hsp.identities}/{hsp.align_length}, "
                                                f"Query Cover: {query_cover:.2f}%, Percent Identity: {percent_identity:.2f}%"
                                ), output_handle, "fasta")

                            saved_alignments += 1
                            break  # Uses only the best HSP per alignment
            
            # Displays the final summary
            
            print(f"Alignments saved for {name_gene}: {saved_alignments}")
        
        print(f"Filtered BLAST results for {name_gene} were saved in '{output_path}'")


In [5]:
genes_names = ["glgA", "glgD", "glgB", "93290188"]
blast_and_filter(genes_names)

Starting BLAST for glgA...
BLAST completed for glgA.
Number of alignments found for glgA: 50
Alignments saved for glgA: 6
Filtered BLAST results for glgA were saved in 'blast_results/glgA_blast.fasta'
Starting BLAST for glgD...
BLAST completed for glgD.
Number of alignments found for glgD: 50
Alignments saved for glgD: 6
Filtered BLAST results for glgD were saved in 'blast_results/glgD_blast.fasta'
Starting BLAST for glgB...
BLAST completed for glgB.
Number of alignments found for glgB: 50
Alignments saved for glgB: 6
Filtered BLAST results for glgB were saved in 'blast_results/glgB_blast.fasta'
Starting BLAST for 93290188...
BLAST completed for 93290188.
Number of alignments found for 93290188: 50
Alignments saved for 93290188: 7
Filtered BLAST results for 93290188 were saved in 'blast_results/93290188_blast.fasta'
